In [1]:
import logging
import tensorflow as tf


GPU_FROM = 3
GPU_TO = 4

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs visible:{len(visible_devices)}")
tf.config.set_visible_devices(visible_devices[GPU_FROM:GPU_TO],'GPU')

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs to be used: {len(visible_devices)}")

2023-01-26 12:53:01.224565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-26 12:53:01.365734: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-26 12:53:01.402222: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-26 12:53:02.063206: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [15]:
import string
import html
import tensorflow_datasets as tfds

from segmentation.model import SpaceSegmentationTransformer
from segmentation.model import LossWithVoids

punc_mapping = {ord(x):x for x in string.punctuation}
entity_mapping = {f" ?{k}": v for k, v in html.entities.html5.items() if k.endswith(";") and v in string.punctuation}
punc_mapping = {f' ?&?#?{k};': v for k, v in punc_mapping.items()}

train, test = tfds.load('ag_news_subset', split="train"), tfds.load('ag_news_subset', split="test")

def unescape(text):
    for match, replace in punc_mapping.items():
        text = tf.strings.regex_replace(text, match, replace)
    for match, replace in entity_mapping.items():#
        text = tf.strings.regex_replace(text, match, replace)
    return text

def join_title_desc(text_dict):
    return text_dict['title'] + ' ' + text_dict['description']

train = train.map(join_title_desc).map(unescape).filter(lambda x: tf.strings.length(x) > 100)
test = test.map(join_title_desc).map(unescape).filter(lambda x: tf.strings.length(x) > 100)

for x in train.take(1):
    print(x)

tf.Tensor(b"AMD Debuts Dual-Core Opteron Processor AMD's new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.", shape=(), dtype=string)


2023-01-26 12:59:42.737204: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [16]:

def generate_labels(text):
    max_chars = 100

    last_char = tf.strings.substr(text, max_chars, max_chars+1)
    text = tf.strings.substr(text,0, max_chars)
    text = tf.strings.lower(text)
    split_on_whitespace = tf.strings.strip(tf.strings.split(text))
    encoder_in = tf.strings.reduce_join(split_on_whitespace, axis=-1)
    
    space_indices = tf.math.cumsum(tf.strings.length(split_on_whitespace)) - 1 # subtract 1 to index from 0
    seq_len = space_indices[-1] + 1
    if not tf.strings.regex_full_match(last_char, " "):
        space_indices = space_indices[:-1]
    encoder_out = tf.ones((max_chars,))
    encoder_out = tf.concat([
        tf.ones((seq_len,)),
        tf.zeros((max_chars - seq_len,))
    ], axis=-1)
    encoder_out = tf.tensor_scatter_nd_update(
        encoder_out, 
        space_indices[...,tf.newaxis], # Expand dims to create a 'list' of indices
        tf.ones_like(space_indices, dtype=encoder_out.dtype)*2) 
    
    return (encoder_in, None), tf.cast(encoder_out, "float16")

train = train.map(generate_labels)
val = test.map(generate_labels)
for x in train.take(1):
    print(x)

((<tf.Tensor: shape=(), dtype=string, numpy=b"amddebutsdual-coreopteronprocessoramd'snewdual-coreopteronchipisdesignedmainlyforcorpo">, None), <tf.Tensor: shape=(100,), dtype=float16, numpy=
array([1., 1., 2., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       1., 1., 1., 1., 2., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 2., 1., 2., 1., 1., 1., 1.,
       1., 1., 1., 2., 1., 1., 1., 1., 1., 2., 1., 1., 2., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float16)>)


2023-01-26 12:59:49.878079: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [17]:
tokenizer = tf.keras.layers.TextVectorization(
    output_sequence_length=100,
    standardize="lower",
    split="character",
    output_mode="int",
)

tokenizer.adapt(train.map(lambda x,y: x[0]).take(1000))
tokenizer.get_vocabulary()

2023-01-26 12:59:56.621126: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


['',
 '[UNK]',
 'e',
 'a',
 's',
 't',
 'o',
 'i',
 'r',
 'n',
 'l',
 'd',
 'c',
 'h',
 'u',
 'p',
 'm',
 'f',
 'g',
 'b',
 'w',
 'y',
 'k',
 'v',
 '-',
 ',',
 '.',
 "'",
 '(',
 ')',
 'x',
 '0',
 'j',
 'q',
 '2',
 '1',
 '&',
 ':',
 'z',
 '3',
 '4',
 '<',
 '"',
 '\\',
 '5',
 '>',
 '/',
 '7',
 '6',
 '$',
 '8',
 '9',
 ';',
 '?',
 '=',
 '!',
 '#',
 '*']

In [18]:
D_MODEL = 512

model = SpaceSegmentationTransformer(
    num_layers=2,
    d_model=D_MODEL,
    num_attention_heads=3,
    seq_len=100,
    dff=1028,
    input_tokenizer=tokenizer,
    dropout_rate=0.1
)

In [19]:
from segmentation.metrics import SparseAccuracyWithIgnore
from segmentation.metrics import SparsePrecision
from segmentation.metrics import SparseRecall
from segmentation.metrics import SparseF1
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=200):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return 2* tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
        SparseAccuracyWithIgnore(ignore_token=0.),
        SparsePrecision(class_id=2, name="space_precision"),
        SparseRecall(class_id=2, name="space_recall"),
        SparseF1(class_id=2, name="space_f1"),
        SparsePrecision(class_id=1, name="char_precision"),
        SparseRecall(class_id=1, name="char_recall"),
        SparseF1(class_id=1, name="char_f1"),
    ]
)

In [20]:
train_ds = train.shuffle(100).batch(8)
val_ds = val.batch(8)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_space_precision",
        patience=100,
        mode="max",
        restore_best_weights=True
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds, 
    epochs=100, 
    callbacks=callbacks, 
    steps_per_epoch=10,
    validation_freq=10
    )

Epoch 1/100
 7/10 [====================>.........] - ETA: 0s - loss: 0.9525 - sparse_categorical_accuracy: 0.6300 - space_precision: 0.1611 - space_recall: 0.0867 - space_f1: 0.1127 - char_precision: 0.7202 - char_recall: 0.7570 - char_f1: 0.7381

2023-01-26 13:00:07.515761: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


10/10 [==============================] - ETA: 0s - loss: 0.8811 - sparse_categorical_accuracy: 0.6492 - space_precision: 0.1760 - space_recall: 0.1523 - space_f1: 0.1633 - char_precision: 0.7197 - char_recall: 0.7660 - char_f1: 0.7421WARNING:tensorflow:Early stopping conditioned on metric `val_space_precision` which is not available. Available metrics are: loss,sparse_categorical_accuracy,space_precision,space_recall,space_f1,char_precision,char_recall,char_f1


10/10 [==============================] - 5s 22ms/step - loss: 0.8811 - sparse_categorical_accuracy: 0.6492 - space_precision: 0.1760 - space_recall: 0.1523 - space_f1: 0.1633 - char_precision: 0.7197 - char_recall: 0.7660 - char_f1: 0.7421
Epoch 2/100
10/10 [==============================] - ETA: 0s - loss: 0.5457 - sparse_categorical_accuracy: 0.7821 - space_precision: 0.2854 - space_recall: 0.2026 - space_f1: 0.2369 - char_precision: 0.7035 - char_recall: 0.9151 - char_f1: 0.7955 WARNING:tensorflow:Early stopping conditioned on metric `val_space_precision` which is not available. Available metrics are: loss,sparse_categorical_accuracy,space_precision,space_recall,space_f1,char_precision,char_recall,char_f1


10/10 [==============================] - 0s 22ms/step - loss: 0.5457 - sparse_categorical_accuracy: 0.7821 - space_precision: 0.2854 - space_recall: 0.2026 - space_f1: 0.2369 - char_precision: 0.7035 - char_recall: 0.9151 - char_f1: 0.7955
Epoch 3/100
10/10 [==============================] - ETA: 0s - loss: 0.4737 - sparse_categorical_accuracy: 0.8047 - space_precision: 0.4073 - space_recall: 0.1427 - space_f1: 0.2114 - char_precision: 0.6907 - char_recall: 0.9557 - char_f1: 0.8019 WARNING:tensorflow:Early stopping conditioned on metric `val_space_precision` which is not available. Available metrics are: loss,sparse_categorical_accuracy,space_precision,space_recall,space_f1,char_precision,char_recall,char_f1


...

10/10 [==============================] - 0s 23ms/step - loss: 0.4873 - sparse_categorical_accuracy: 0.8102 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6789 - char_recall: 1.0000 - char_f1: 0.8087
Epoch 97/100
10/10 [==============================] - ETA: 0s - loss: 0.4771 - sparse_categorical_accuracy: 0.8167 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6879 - char_recall: 1.0000 - char_f1: 0.8151WARNING:tensorflow:Early stopping conditioned on metric `val_space_precision` which is not available. Available metrics are: loss,sparse_categorical_accuracy,space_precision,space_recall,space_f1,char_precision,char_recall,char_f1


10/10 [==============================] - 0s 22ms/step - loss: 0.4771 - sparse_categorical_accuracy: 0.8167 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6879 - char_recall: 1.0000 - char_f1: 0.8151
Epoch 98/100
10/10 [==============================] - ETA: 0s - loss: 0.4728 - sparse_categorical_accuracy: 0.8199 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6925 - char_recall: 1.0000 - char_f1: 0.8183WARNING:tensorflow:Early stopping conditioned on metric `val_space_precision` which is not available. Available metrics are: loss,sparse_categorical_accuracy,space_precision,space_recall,space_f1,char_precision,char_recall,char_f1


10/10 [==============================] - 0s 22ms/step - loss: 0.4728 - sparse_categorical_accuracy: 0.8199 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6925 - char_recall: 1.0000 - char_f1: 0.8183
Epoch 99/100
10/10 [==============================] - ETA: 0s - loss: 0.4804 - sparse_categorical_accuracy: 0.8145 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6856 - char_recall: 1.0000 - char_f1: 0.8135WARNING:tensorflow:Early stopping conditioned on metric `val_space_precision` which is not available. Available metrics are: loss,sparse_categorical_accuracy,space_precision,space_recall,space_f1,char_precision,char_recall,char_f1


10/10 [==============================] - 0s 22ms/step - loss: 0.4804 - sparse_categorical_accuracy: 0.8145 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6856 - char_recall: 1.0000 - char_f1: 0.8135
Epoch 100/100
10/10 [==============================] - 11s 1s/step - loss: 0.4758 - sparse_categorical_accuracy: 0.8175 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6895 - char_recall: 1.0000 - char_f1: 0.8162 - val_loss: 0.4798 - val_sparse_categorical_accuracy: 0.8161 - val_space_precision: 0.0000e+00 - val_space_recall: 0.0000e+00 - val_space_f1: nan - val_char_precision: 0.6873 - val_char_recall: 1.0000 - val_char_f1: 0.8147


2023-01-26 13:02:22.257033: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [21]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
second_ax = ax.twinx()

ax.plot(history.history['loss'], label="loss")
second_ax.plot(history.history['space_recall'], label="space recall", color="red")
fig.legend()

In [22]:
model.evaluate(val_ds)

949/949 [==============================] - 13s 13ms/step - loss: 0.4798 - sparse_categorical_accuracy: 0.8161 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6873 - char_recall: 1.0000 - char_f1: 0.8147


[0.4797550439834595,
 0.8160789608955383,
 0.0,
 0.0,
 nan,
 0.6873379945755005,
 1.0,
 0.8147010207176208]

In [23]:
preds = model([("dunningkruger", "eeeeeeeeeeeeeeeeeeeee", "thequickbrownfoxjumpedoverthelazydog"),(None,None, None)])
preds, tf.argmax(preds, axis=-1)

(<tf.Tensor: shape=(3, 100, 3), dtype=float32, numpy=
 array([[[-16.99924   ,   2.0721548 ,   0.4026579 ],
         [-16.99762   ,   2.0717075 ,   0.40232304],
         [-16.999203  ,   2.0721197 ,   0.4026194 ],
         [-16.999203  ,   2.0721197 ,   0.4026194 ],
         [-16.997372  ,   2.0716274 ,   0.4022716 ],
         [-16.999218  ,   2.0721533 ,   0.40265852],
         [-16.999245  ,   2.0721548 ,   0.40265906],
         [-16.999252  ,   2.0721607 ,   0.4026651 ],
         [-16.99762   ,   2.0717072 ,   0.4023228 ],
         [-16.99762   ,   2.071707  ,   0.40232238],
         [-16.999245  ,   2.0721548 ,   0.402659  ],
         [-16.996853  ,   2.0714152 ,   0.4020655 ],
         [-16.99762   ,   2.0717072 ,   0.4023228 ],
         [-17.01897   ,   2.077931  ,   0.40708598],
         [-17.016283  ,   2.0774305 ,   0.406688  ],
         [-17.019657  ,   2.0782142 ,   0.40728968],
         [-17.019958  ,   2.0783627 ,   0.40753227],
         [-17.019958  ,   2.0783627 ,   0.407

In [24]:
model.encoder.pos_embedding(tokenizer(["e"]))

<tf.Tensor: shape=(1, 100, 512), dtype=float32, numpy=
array([[[-0.8135057 ,  2.4654276 ,  1.954949  , ...,  0.56842756,
          0.9597004 ,  3.7965462 ],
        [ 0.5219509 , -0.16743678,  0.6210165 , ...,  1.9315889 ,
          0.2138592 ,  0.71559334],
        [ 0.58977735, -0.05287826,  0.7771991 , ...,  1.9315889 ,
          0.2138592 ,  0.71559334],
        ...,
        [ 0.06008768, -1.6146605 ,  0.5641642 , ...,  1.9315305 ,
          0.2138049 ,  0.7155428 ],
        [-0.89290196, -0.70424247, -0.27069739, ...,  1.9315293 ,
          0.21380377,  0.7155417 ],
        [-1.3187269 , -0.03914171, -1.0332861 , ...,  1.9315281 ,
          0.21380264,  0.7155407 ]]], dtype=float32)>

In [25]:
model.encoder.pos_embedding(tokenizer(["d"]))

<tf.Tensor: shape=(1, 100, 512), dtype=float32, numpy=
array([[[ 1.4004881 ,  0.32033703,  0.76646286, ...,  0.3870238 ,
         -0.3503027 , -1.6323583 ],
        [ 0.5219509 , -0.16743678,  0.6210165 , ...,  1.9315889 ,
          0.2138592 ,  0.71559334],
        [ 0.58977735, -0.05287826,  0.7771991 , ...,  1.9315889 ,
          0.2138592 ,  0.71559334],
        ...,
        [ 0.06008768, -1.6146605 ,  0.5641642 , ...,  1.9315305 ,
          0.2138049 ,  0.7155428 ],
        [-0.89290196, -0.70424247, -0.27069739, ...,  1.9315293 ,
          0.21380377,  0.7155417 ],
        [-1.3187269 , -0.03914171, -1.0332861 , ...,  1.9315281 ,
          0.21380264,  0.7155407 ]]], dtype=float32)>

In [26]:
import lovely_tensors as lt
lt.monkey_patch()
[print(x) for x in model.trainable_variables]

<tf.Variable 'space_segmentation_transformer/encoder/postional_embedding/embedding/embeddings:0' shape=(58, 512) dtype=float32, numpy=
array([[-0.01412093, -0.04372099, -0.00799673, ...,  0.0411708 ,
        -0.03474284, -0.01256912],
       [-0.02968057, -0.073121  , -0.08130331, ...,  0.03610411,
         0.01929422,  0.05080995],
       [-0.03595221,  0.10895754,  0.08639736, ..., -0.01907299,
        -0.00178101,  0.12359105],
       ...,
       [ 0.00896447, -0.01703224, -0.00899448, ..., -0.04177563,
        -0.0552373 ,  0.03061399],
       [-0.05058952,  0.02026631, -0.04149869, ..., -0.02427576,
        -0.07221463, -0.02539273],
       [-0.04887834,  0.01469706, -0.03826246, ..., -0.06998698,
        -0.0172425 , -0.01516993]], dtype=float32)>
<tf.Variable 'space_segmentation_transformer/encoder/encoder_sublayer_0/multi_head_attention/query/kernel:0' shape=(512, 3, 512) dtype=float32, numpy=
array([[[-7.43764918e-04,  2.94043142e-02,  1.38004776e-02, ...,
          1.84714645

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]